# 02 Tiled Matmul**FreeCodeCamp CUDA Course - Module 5**Original Course: [https://www.youtube.com/watch?v=86FAWCzIe_4](https://www.youtube.com/watch?v=86FAWCzIe_4)Source File: `02_tiled_matmul.cu`---## OverviewImplement matrix multiplication using CUDA.---

## Learning ObjectivesBy the end of this notebook, you will:1. Understand CUDA kernel syntax and execution2. Learn GPU memory allocation and data transfer3. Use shared memory for optimization---

## SetupMake sure you've completed the setup from the first notebook (GPU enabled, nvcc4jupyter installed).---

## Key Concepts- **Kernel Function**: Uses `__global__` qualifier for GPU execution- **Shared Memory**: Fast on-chip memory shared by threads in a block- **Device Memory**: Allocated using `cudaMalloc`- **Kernel Launch**: Syntax `kernel<<<blocks, threads>>>(...)`- **Synchronization**: `cudaDeviceSynchronize()` waits for GPU completion- **Block Synchronization**: `__syncthreads()` synchronizes threads in a block---## CUDA Implementation

In [ ]:
%%cu#include <cuda_runtime.h>#include <iostream>#define TILE_SIZE 16__global__ void matrixMultiplyOptimized(float* A, float* B, float* C, int M, int N, int K) {    __shared__ float sharedA[TILE_SIZE][TILE_SIZE];    __shared__ float sharedB[TILE_SIZE][TILE_SIZE];        int bx = blockIdx.x, by = blockIdx.y;    int tx = threadIdx.x, ty = threadIdx.y;        int row = by * TILE_SIZE + ty;    int col = bx * TILE_SIZE + tx;        float sum = 0.0f;        for (int tile = 0; tile < (K + TILE_SIZE - 1) / TILE_SIZE; ++tile) {        if (row < M && tile * TILE_SIZE + tx < K)            sharedA[ty][tx] = A[row * K + tile * TILE_SIZE + tx];        else            sharedA[ty][tx] = 0.0f;                if (col < N && tile * TILE_SIZE + ty < K)            sharedB[ty][tx] = B[(tile * TILE_SIZE + ty) * N + col];        else            sharedB[ty][tx] = 0.0f;                __syncthreads();                for (int k = 0; k < TILE_SIZE; ++k)            sum += sharedA[ty][k] * sharedB[k][tx];                __syncthreads();    }        if (row < M && col < N)        C[row * N + col] = sum;}int main() {    // Define matrix dimensions    const int M = 1024; // Number of rows in A and C    const int N = 1024; // Number of columns in B and C    const int K = 1024; // Number of columns in A and rows in B    // Calculate matrix sizes in bytes    size_t size_A = M * K * sizeof(float);    size_t size_B = K * N * sizeof(float);    size_t size_C = M * N * sizeof(float);    // Declare device pointers    float *d_A, *d_B, *d_C;    // Allocate device memory    cudaMalloc(&d_A, size_A);    cudaMalloc(&d_B, size_B);    cudaMalloc(&d_C, size_C);    // Kernel launch code    dim3 blockDim(TILE_SIZE, TILE_SIZE);    dim3 gridDim((N + TILE_SIZE - 1) / TILE_SIZE, (M + TILE_SIZE - 1) / TILE_SIZE);    matrixMultiplyOptimized<<<gridDim, blockDim>>>(d_A, d_B, d_C, M, N, K);    // Synchronize device    cudaDeviceSynchronize();    // Free device memory    cudaFree(d_A);    cudaFree(d_B);    cudaFree(d_C);    // Check for any CUDA errors    cudaError_t error = cudaGetLastError();    if (error != cudaSuccess) {        std::cerr << "CUDA error: " << cudaGetErrorString(error) << std::endl;        return -1;    }    return 0;}

## ExercisesExperiment with these modifications:1. **Matrix Sizes**: Try different M, N, K dimensions   - What happens with non-square matrices?   - Test with very large matrices2. **Verify Correctness**: Add code to verify the result against CPU computation3. **Block Size Impact**: Experiment with different BLOCK_SIZE values   - Measure performance for each4. **Measure Throughput**: Calculate FLOPS (floating-point operations per second)

---## Key Takeaways- CUDA enables massive parallelism for compute-intensive tasks- Proper memory management is crucial for performance- Understanding the thread hierarchy helps write efficient kernels- Always synchronize when needed to ensure correctness---## Next StepsContinue to the next notebook in Module 5 to learn more CUDA concepts!---## Notes*Use this space for your learning notes:*